<a href="https://colab.research.google.com/github/y001003/system_quality_project/blob/main/notebooks/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회

https://dacon.io/competitions/official/235687/overview/description

## 1. Library Import & Data 불러오기

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 82 kB/s 


In [ ]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import catboost as cb
import lightgbm as lgb
from tqdm import tqdm

In [ ]:
PATH = '/content/drive/MyDrive/project/data/system_quality_customer_complain_data/'
train_err  = pd.read_csv(PATH+'train_err_data_.csv')
id_error = train_err[['user_id','errtype']].values
# 불만을 제기한 사람 타겟값 생성
train_prob = pd.read_csv(PATH+'train_problem_data_.csv')

## 2. 학습 데이터 생성

### 2_1 train_df 생성

In [ ]:
#error 0으로 이루어진 데이터셋 생성
error = np.zeros((15000,42)) # 15000개 user_id, 41가지(maxnum 42) errtype 

for person_idx, err in tqdm(id_error):
    # (person_idx - 10000)에 해당하는 행에, 열 기준 errtype 값의 -1 에 해당하는 위치에 error값을 +1
    error[person_idx - 10000,err - 1] += 1
error 

100%|██████████| 16554663/16554663 [00:49<00:00, 332163.93it/s]


array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [ ]:
train_df = pd.DataFrame(error)
train_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,...,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,...,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


### 2_2 target 생성

In [ ]:
# 0 으로 이루어진 열 생성
problem = np.zeros(15000)
# user_id 각각에 10000 빼서 저장
user_ids = train_prob['user_id'].unique() - 10000
# probelm[유저id번호] 에 1 저장
problem[user_ids] = 1

display(problem, user_ids, sum(problem))

array([0., 1., 0., ..., 1., 1., 0.])

array([ 9224, 13664,  5166, ...,  9114, 11505,  8822])

5000.0

In [ ]:
target = pd.DataFrame(problem)
target

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
14995,0.0
14996,0.0
14997,1.0
14998,1.0


## 3. Modeling


### 3.1 훈련세트 나누기

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score ,f1_score, roc_auc_score, classification_report

In [ ]:
def get_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print(f'\n정확도:: {accuracy:.4f}')
    print(f'정밀도: {precision:.4f}')
    print(f'재현율: {recall:.4f}')
    print(f'F1: {F1:.4f}')
    print(f'AUC: {AUC:.4f}')
    print(classification_report(y_test, y_pred))



In [ ]:
# train validation 분리
X_train, X_val, y_train, y_val = train_test_split(train_df,target,test_size=0.2, random_state=42)
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((12000, 42), (3000, 42), (12000, 1), (3000, 1))

In [ ]:
y_train

,0
9839,0.0
9680,0.0
7093,0.0
11293,0.0
820,0.0
...,...
5191,1.0
13418,1.0
5390,1.0
860,1.0


### 3.2 모델별 학습

#### 3.2.1 랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=10, max_depth=10, min_samples_leaf=8,
                           min_samples_split=20, random_state=0)

model.fit(X_train, y_train)

pred = model.predict(X_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [ ]:
get_eval(y_val,pred)

오차행렬:
 [[1550   83]
 [ 808  559]]

정확도:: 0.7030
정밀도: 0.8707
재현율: 0.4089
F1: 0.5565
AUC: 0.6790
              precision    recall  f1-score   support

         0.0       0.66      0.95      0.78      1633
         1.0       0.87      0.41      0.56      1367

    accuracy                           0.70      3000
   macro avg       0.76      0.68      0.67      3000
weighted avg       0.75      0.70      0.68      3000



#### 3.2.2 GradientBoosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_model2 = GradientBoostingClassifier(n_estimators = 41,
                                   learning_rate = 0.1483,
                                    #loss='log_loss', 
                                    # learning_rate=0.1, 
                                    # n_estimators=100, 
                                    subsample=0.95, 
                                    # criterion='friedman_mse', 
                                    # min_samples_split=2, 
                                    # min_samples_leaf=1, 
                                    # min_weight_fraction_leaf=0.0, 
                                    max_depth=3, 
                                    # min_impurity_decrease=0.0, 
                                    # init=None, 
                                    # random_state=None, 
                                    # max_features=None, 
                                    # verbose=0, 
                                    # max_leaf_nodes=None, 
                                    # warm_start=False, 
                                    # validation_fraction=0.1, 
                                    # n_iter_no_change=None, 
                                    # tol=0.0001, 
                                    # ccp_alpha=0.0
                                    )

model2 = gb_model2.fit(X_train,y_train)
# Validation 데이터셋 예상
pred2 = model2.predict(X_val)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
get_eval(y_val,pred2)

오차행렬:
 [[1551   82]
 [ 798  569]]

정확도:: 0.7067
정밀도: 0.8740
재현율: 0.4162
F1: 0.5639
AUC: 0.6830
              precision    recall  f1-score   support

         0.0       0.66      0.95      0.78      1633
         1.0       0.87      0.42      0.56      1367

    accuracy                           0.71      3000
   macro avg       0.77      0.68      0.67      3000
weighted avg       0.76      0.71      0.68      3000



#### 3.2.3 CatBoost

In [ ]:
cb_dtrain = cb.Pool(data=X_train, label = y_train)
cb_deval = cb.Pool(data=X_val, label = y_val)
cb_param = {
    'max_depth':10, # 트리 깊이
    'learning_rate': 0.01, # 학습률
    'n_estimators': 100, # 트리 생성 갯수
    'eval_metric' : 'Accuracy', # 평가 척도
    'loss_function': 'MultiClass' # 손실 함수
}
cb_model3 = cb.CatBoostClassifier(iterations=30,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')

model3 = cb_model3.fit(cb_dtrain)

In [ ]:
model3 = cb.CatBoostClassifier(iterations=2,
                           learning_rate=0.01,
                           max_depth=10,
                           early_stopping_rounds=3
                           )
cb_model3 = model3.fit(X_train,y_train)

In [ ]:
pred3 = model3.predict(X_val)
get_eval(y_val,pred3)

오차행렬:
 [[1531  102]
 [ 743  624]]

정확도:: 0.7183
정밀도: 0.8595
재현율: 0.4565
F1: 0.5963
AUC: 0.6970
              precision    recall  f1-score   support

         0.0       0.67      0.94      0.78      1633
         1.0       0.86      0.46      0.60      1367

    accuracy                           0.72      3000
   macro avg       0.77      0.70      0.69      3000
weighted avg       0.76      0.72      0.70      3000



#### 3.2.4 Xgboost

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
d_train = xgb.DMatrix(X_train,y_train)
d_val = xgb.DMatrix(X_val, y_val) 

params = {
    'max_depth':5,
    'eta': 0.01,
    'booster' : 'gbtree',
    'eval_metric' : 'logloss',
}
num_rounds = 500
wlist = [(d_train,'train'),(d_val,'eval')]
# model4 = xgb.train(params = params , dtrain = d_train, num_boost_round= num_rounds ,evals= wlist,early_stopping_rounds=10,verbose_eval= 0)
model4 = XGBClassifier(max_depth=5,learning_rate=0.01,booster='gbtree')
model4.fit(X_train,y_train)
pred_probs = model4.predict(X_val)
pred4 = np.where(pred_probs > 0.5 , 1, 0)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
Exception ignored in: <function Booster.__del__ at 0x7f7016076170>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/xgboost/core.py", line 957, in __del__
    if self.handle is not None:
AttributeError: 'Booster' object has no attribute 'handle'


In [ ]:
get_eval(y_val,pred4)

오차행렬:
 [[1567   66]
 [ 851  516]]

정확도:: 0.6943
정밀도: 0.8866
재현율: 0.3775
F1: 0.5295
AUC: 0.6685
              precision    recall  f1-score   support

         0.0       0.65      0.96      0.77      1633
         1.0       0.89      0.38      0.53      1367

    accuracy                           0.69      3000
   macro avg       0.77      0.67      0.65      3000
weighted avg       0.76      0.69      0.66      3000



### 3.3 앙상블

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
models = [
    ('gbc', GradientBoostingClassifier(n_estimators = 41,
                                   learning_rate = 0.1483,
                                    #loss='log_loss', 
                                    # learning_rate=0.1, 
                                    # n_estimators=100, 
                                    subsample=0.95, 
                                    # criterion='friedman_mse', 
                                    # min_samples_split=2, 
                                    # min_samples_leaf=1, 
                                    # min_weight_fraction_leaf=0.0, 
                                    max_depth=3, 
                                    # min_impurity_decrease=0.0, 
                                    # init=None, 
                                    # random_state=None, 
                                    # max_features=None, 
                                    # verbose=0, 
                                    # max_leaf_nodes=None, 
                                    # warm_start=False, 
                                    # validation_fraction=0.1, 
                                    # n_iter_no_change=None, 
                                    # tol=0.0001, 
                                    # ccp_alpha=0.0
                                    )),
    ('rfc', RandomForestClassifier(n_estimators=10, max_depth=10, min_samples_leaf=8,
                           min_samples_split=20, random_state=0)),
    ('xgb', XGBClassifier(max_depth=5,learning_rate=0.01,booster='gbtree')),
    ('cb', cb.CatBoostClassifier(iterations=2,
                           learning_rate=0.01,
                           max_depth=10,
                           early_stopping_rounds=3
                           )),
]

In [ ]:
# hard vote
hard_vote  = VotingClassifier(models, voting='hard')
hard_vote.fit(X_train, y_train)

# soft vote
soft_vote  = VotingClassifier(models, voting='soft')
soft_vote.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0:	learn: 0.6886158	total: 112ms	remaining: 112ms
1:	learn: 0.6840796	total: 203ms	remaining: 0us


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0:	learn: 0.6886158	total: 81ms	remaining: 81ms
1:	learn: 0.6840796	total: 159ms	remaining: 0us


VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(learning_rate=0.1483,
                                                         n_estimators=41,
                                                         subsample=0.95)),
                             ('rfc',
                              RandomForestClassifier(max_depth=10,
                                                     min_samples_leaf=8,
                                                     min_samples_split=20,
                                                     n_estimators=10,
                                                     random_state=0)),
                             ('xgb',
                              XGBClassifier(learning_rate=0.01, max_depth=5)),
                             ('cb',
                              <catboost.core.CatBoostClassifier object at 0x7f6fa70afc50>)],
                 voting='soft')

#### 3.3.1 HardVote

In [ ]:
en_pred = hard_vote.predict(X_val)
get_eval(y_val,en_pred)

오차행렬:
 [[1570   63]
 [ 855  512]]

정확도:: 0.6940
정밀도: 0.8904
재현율: 0.3745
F1: 0.5273
AUC: 0.6680
              precision    recall  f1-score   support

         0.0       0.65      0.96      0.77      1633
         1.0       0.89      0.37      0.53      1367

    accuracy                           0.69      3000
   macro avg       0.77      0.67      0.65      3000
weighted avg       0.76      0.69      0.66      3000



#### 3.3.2 softvote

In [ ]:
en_pred2 = soft_vote.predict(X_val)
get_eval(y_val,en_pred2)

오차행렬:
 [[1563   70]
 [ 820  547]]

정확도:: 0.7033
정밀도: 0.8865
재현율: 0.4001
F1: 0.5514
AUC: 0.6786
              precision    recall  f1-score   support

         0.0       0.66      0.96      0.78      1633
         1.0       0.89      0.40      0.55      1367

    accuracy                           0.70      3000
   macro avg       0.77      0.68      0.66      3000
weighted avg       0.76      0.70      0.67      3000



#### 3.3.3 StackNetClassifier

In [ ]:
!git clone https://gitlab.com/YannBerthelot/kaggle_pystacknet.git
# print(os.listdir("kaggle_pystacknet/pystacknet"))
!pip install "kaggle_pystacknet/pystacknet"

fatal: destination path 'kaggle_pystacknet' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./kaggle_pystacknet/pystacknet
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for pystacknet: filename=pystacknet-0.0.1-py3-none-any.whl size=21925 sha256=dcb52c0bbad8f2f9cd9fa73d39a79251aa5c8fd92f3d575508d1bdd8e73f310b
  Stored in directory: /tmp/pip-ephem-wheel-cache-u9g7plp3/wheels/91/4b/a2/adc9ddd4368da9824c1b5e80a2179e66510246486ecbfcf953
Successfully built pystacknet


In [ ]:
!pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from pystacknet.pystacknet import StackNetClassifier

In [ ]:
clf1 = GradientBoostingClassifier(n_estimators = 41,
                                   learning_rate = 0.1483,
                                    #loss='log_loss', 
                                    # learning_rate=0.1, 
                                    # n_estimators=100, 
                                    subsample=0.95, 
                                    # criterion='friedman_mse', 
                                    # min_samples_split=2, 
                                    # min_samples_leaf=1, 
                                    # min_weight_fraction_leaf=0.0, 
                                    max_depth=3, 
                                    # min_impurity_decrease=0.0, 
                                    # init=None, 
                                    # random_state=None, 
                                    # max_features=None, 
                                    # verbose=0, 
                                    # max_leaf_nodes=None, 
                                    # warm_start=False, 
                                    # validation_fraction=0.1, 
                                    # n_iter_no_change=None, 
                                    # tol=0.0001, 
                                    # ccp_alpha=0.0
                                    )
clf2 = RandomForestClassifier(n_estimators=10, max_depth=10, min_samples_leaf=8,
                           min_samples_split=20, random_state=0)
clf3 = XGBClassifier(max_depth=5,learning_rate=0.01,booster='gbtree')
clf4 = cb.CatBoostClassifier(iterations=2,
                           learning_rate=0.01,
                           max_depth=10,
                           early_stopping_rounds=3
                           )
models = [[clf1, clf2, clf3,clf4], # Level 1
          [clf4]] # Level 2
# Specify parameters for stacked model and begin training
model = StackNetClassifier(models, 
                           metric="auc", 
                           folds=3,
                           restacking=False,
                           use_retraining=True,
                           use_proba=True, # To use predict_proba after training
                           random_state=42,
                           n_jobs=1,                        
                           verbose=1)

# Fit the entire model tree
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/pystacknet/pystacknet.py:905: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


====================== Start of Level 0 ======================
Input Dimensionality 42 at Level 0 
4 models included in Level 0 
0:	learn: 0.6885021	total: 67ms	remaining: 67ms
1:	learn: 0.6840659	total: 127ms	remaining: 0us
Fold 1/3 , model 0 , auc===0.797469 
Fold 1/3 , model 1 , auc===0.796640 
Fold 1/3 , model 2 , auc===0.784307 
Fold 1/3 , model 3 , auc===0.753718 
=========== end of fold 1 in level 0 ===========
0:	learn: 0.6886396	total: 59.8ms	remaining: 59.8ms
1:	learn: 0.6842384	total: 119ms	remaining: 0us
Fold 2/3 , model 0 , auc===0.801772 
Fold 2/3 , model 1 , auc===0.792994 
Fold 2/3 , model 2 , auc===0.792880 
Fold 2/3 , model 3 , auc===0.772710 
=========== end of fold 2 in level 0 ===========
0:	learn: 0.6886067	total: 62.4ms	remaining: 62.4ms
1:	learn: 0.6846851	total: 123ms	remaining: 0us
Fold 3/3 , model 0 , auc===0.795980 
Fold 3/3 , model 1 , auc===0.782297 
Fold 3/3 , model 2 , auc===0.789830 
Fold 3/3 , model 3 , auc===0.761170 
=========== end of fold 3 in leve

In [ ]:
model.predict_proba(X_val)[:,1]

====================== Start of Level 0 ======================
1 estimators included in Level 0 


ValueError: ignored

In [ ]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
9839,0.0,0.0,4.0,388.0,10.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9680,0.0,0.0,0.0,477.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7093,0.0,0.0,0.0,2.0,464.0,0.0,1.0,0.0,0.0,4.0,...,35.0,4.0,0.0,1.0,1.0,1.0,0.0,149.0,68.0,0.0
11293,0.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0,...,14.0,253.0,1.0,1.0,1.0,0.0,0.0,148.0,8.0,3.0
820,0.0,0.0,0.0,0.0,53.0,0.0,0.0,0.0,0.0,0.0,...,14.0,0.0,0.0,1.0,1.0,0.0,0.0,94.0,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5191,0.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,0.0,1.0,...,39.0,1.0,4.0,0.0,0.0,0.0,0.0,43.0,1.0,1.0
13418,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,...,17.0,4.0,3.0,2.0,2.0,0.0,0.0,14.0,0.0,3.0
5390,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,32.0,0.0,1.0,2.0,2.0,0.0,0.0,14.0,0.0,0.0
860,0.0,0.0,0.0,20.0,124.0,0.0,1.0,0.0,0.0,0.0,...,36.0,0.0,0.0,1.0,1.0,0.0,0.0,42.0,0.0,2.0
